In [1]:
import pandas as pd
import os
from glob import glob
import numpy as np
from pandas.api.types import CategoricalDtype

In [3]:
ex1 = pd.read_parquet('test/4.청구입금정보/2018_청구정보_전체.parquet')

In [4]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '대표결제일', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_포인트_월적립_B0M', '포인트_포인트_월적립_R3M', '포인트_적립포인트_R12M', '포인트_적립포인트_R3M', '포인트_이용포인트_R12M', '포인트_이용포인트_R3M', '포인트_잔여포인트_B0M', '마일_적립포인트_R12M', '마일_적립포인트_R3M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', '마일_잔여포인트_B0M', '할인건수_R3M', '할인금액_R3M', '할인건수_B0M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '선결제건수_R6M', '선결제건수_R3M', '연체건수_R6M', '연체건수_R3M', '혜택수혜금액_R3M', '포인트_마일리지_환산_B0M', '혜택수혜금액']

문자형 컬럼 이름:
['ID', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '할인건수_R3M', '할인건수_B0M']

숫자형 컬럼 이름:
['기준년월', '대표결제일', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '

In [5]:
cols = ['대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '할인건수_R3M', '할인건수_B0M']
print(ex1[cols].head())

  대표결제방법코드 대표청구지고객주소구분코드 대표청구서수령지구분코드 청구서수령방법 할인건수_R3M 할인건수_B0M
0     자동이체           미확인          이메일     이메일    1회 이상    1회 이상
1     자동이체           미확인    당사페이앱+이메일   문자메세지    1회 이상    1회 이상
2     자동이체           미확인          이메일     이메일    1회 이상    1회 이상
3     자동이체           주거지           우편      우편    1회 이상    1회 이상
4     자동이체           주거지           우편      우편    1회 이상    1회 이상


In [6]:
print("할인건수_R3M unique values:")
print(ex1['할인건수_R3M'].unique())

print("\n할인건수_B0M unique values:")
print(ex1['할인건수_B0M'].unique())

할인건수_R3M unique values:
['1회 이상' '10회 이상' '20회 이상' '30회 이상' '40회 이상']

할인건수_B0M unique values:
['1회 이상' '10회 이상']


In [7]:
count_cols2 = ['할인건수_R3M', '할인건수_B0M']

for col in count_cols2:
    # 1) 문자열로 바꾼 뒤 “회” 제거
    ex1[col] = ex1[col].astype(str).str.replace('회 이상', '', regex=False)
    
    # 2) 숫자로 변환, 변환 불가(예: 'nan', 'None')는 NaN → fillna(-1) → 정수형
    ex1[col] = (
        pd.to_numeric(ex1[col], errors='coerce')
          .fillna(-1)
          .astype(int)
    )

In [8]:
le_cols = ['대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법']

for col in le_cols:
    codes, uniques = pd.factorize(ex1[col], sort=True)
    ex1[col] = codes


In [10]:
ex1

,기준년월,ID,대표결제일,대표결제방법코드,대표청구지고객주소구분코드,대표청구서수령지구분코드,청구서수령방법,청구서발송여부_B0,청구서발송여부_R3M,청구서발송여부_R6M,...,할인금액_청구서_B0M,상환개월수_결제일_R6M,상환개월수_결제일_R3M,선결제건수_R6M,선결제건수_R3M,연체건수_R6M,연체건수_R3M,혜택수혜금액_R3M,포인트_마일리지_환산_B0M,혜택수혜금액
0,201807,TEST_00000,25,0,0,5,5,1,1,1,...,0,5,2,0,0,0,0,143,148,56
1,201807,TEST_00001,13,0,0,2,2,1,1,1,...,14,6,3,0,0,0,0,0,0,0
2,201807,TEST_00002,25,0,0,5,5,1,1,1,...,50,6,3,0,0,0,0,148,0,50
3,201807,TEST_00003,25,0,1,4,4,1,1,1,...,0,2,1,0,0,4,2,3,0,1
4,201807,TEST_00004,27,0,1,4,4,1,1,1,...,0,6,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,201812,TEST_99995,20,0,2,4,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
599996,201812,TEST_99996,20,0,1,4,4,1,1,1,...,50,5,3,3,2,0,0,149,0,49
599997,201812,TEST_99997,15,0,0,5,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
599998,201812,TEST_99998,12,0,0,0,0,1,1,1,...,242,1,1,7,3,0,0,0,0,0


In [11]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '대표결제일', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_포인트_월적립_B0M', '포인트_포인트_월적립_R3M', '포인트_적립포인트_R12M', '포인트_적립포인트_R3M', '포인트_이용포인트_R12M', '포인트_이용포인트_R3M', '포인트_잔여포인트_B0M', '마일_적립포인트_R12M', '마일_적립포인트_R3M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', '마일_잔여포인트_B0M', '할인건수_R3M', '할인금액_R3M', '할인건수_B0M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '선결제건수_R6M', '선결제건수_R3M', '연체건수_R6M', '연체건수_R3M', '혜택수혜금액_R3M', '포인트_마일리지_환산_B0M', '혜택수혜금액']

문자형 컬럼 이름:
['ID']

숫자형 컬럼 이름:
['기준년월', '대표결제일', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_

In [13]:
cols = ['기준년월',
 'ID',
 '대표결제일',
 '대표청구지고객주소구분코드',
 '대표청구서수령지구분코드',
 '청구서발송여부_B0',
 '청구금액_B0',
 '포인트_마일리지_건별_B0M',
 '포인트_포인트_건별_B0M',
 '포인트_마일리지_월적립_B0M',
 '포인트_포인트_월적립_B0M',
 '포인트_잔여포인트_B0M',
 '할인건수_R3M',
 '할인금액_R3M',
 '할인건수_B0M',
 '선결제건수_R6M',
 '연체건수_R6M']
ex1 = ex1[cols]

In [14]:
ex1.to_parquet('청구_전처리_test.parquet', index=False)